In [77]:
import re
import pickle
import pandas as pd
import numpy as np
from sentence_transformers import util
import torch
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
tqdm.pandas()
import sklearn
sklearn.set_config(transform_output="pandas")
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

In [2]:
if torch.cuda.is_available():
    print("CUDA доступна!")
    device = torch.device("cuda")
else:
    print("CUDA недоступна. Вычисления будут выполняться на CPU.")
    device = torch.device("cpu")

CUDA доступна!


In [3]:
df_load = pd.read_csv('../parsing/files/movie_data.csv').set_index('Unnamed: 0')

/tmp/ipykernel_25501/1241434154.py:1: DtypeWarning: Columns (3,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_load = pd.read_csv('../parsing/files/movie_data.csv').set_index('Unnamed: 0')


In [4]:
my_imputer = ColumnTransformer(
    transformers = [
        ('imputer_zeros', SimpleImputer(strategy='constant', fill_value='Unknown'), ['img', 'actors']),
    ],
    verbose_feature_names_out = False,
    remainder = 'passthrough'
)

In [5]:
df = my_imputer.fit_transform(df_load).reset_index(drop=True)

In [6]:
# df.to_csv('../parsing/files/movie_data_2.csv')

In [7]:
df.columns

Index(['img', 'actors', 'title', 'url', 'time_sec', 'genre', 'director',
       'description', 'year', 'ужасы', 'фэнтези', 'вестерн', 'семейный',
       'мультфильмы', 'приключения', 'военный', 'фантастика', 'драма',
       'комедия', 'мелодрама', 'боевик', 'триллер', 'музыкальный',
       'документальный', 'короткометражный', 'криминал', 'эротика', 'детектив',
       'детский', 'мюзикл', 'аниме', 'биография', 'спорт', 'исторический',
       'title_full'],
      dtype='object')

In [8]:
df['all_text'] = df.apply(lambda row: f'{row['title']} {row['genre']} {row['director']} {row['actors']} {row['description']}', axis=1)
df['all_text'][0]

'Человек-паук: Нет пути домой боевик, приключения, фантастика Джон Уоттс Том Холланд, Зендая, Бенедикт Камбербэтч, Мариса Томей, Джейкоб Баталон, Уиллем Дефо, Альфред Молина, Джейми Фокс, Рис Иванс, Томас Хейден Черч, Джон Фавро, Тоби Магуайр, Эндрю Гарфилд, Тони Револори, Дж. К. Симмонс, Бенедикт Вонг, Энгаури Райс, Мартин Старр, Дж.Б. Смув, Гарри Холланд, Хэннибал Бересс, Паула Ньюсом, Хорхе Лендеборг мл., Чарли Кокс Теперь, когда весь мир узнал, кто скрывается за маской Человека-паука, жизнь Питера Паркера и его близких меняется не в лучшую сторону. Не придумав ничего лучше, Паркер обращается за помощью к Доктору Стрэнджу. Но в процессе заклинания, которое должно было заставить человечество забыть о Питере, что-то идет не так, в результате чего открывается портал в Мультивселенную. Герою предстоит познакомиться со всеми суперзлодеями, с которыми приходилось бороться Человеку-пауку в альтернативных реальностях.  «Человек-паук: Нет пути домой» — супергеройский фильм, основанный на сер

In [9]:
max_length = 512
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2', truncation=True, max_length=max_length)
model = AutoModel.from_pretrained('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2').to(device).half()

In [10]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def get_sentence_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True).to(device)
    with torch.amp.autocast('cuda'):
        with torch.no_grad():
            outputs = model(**inputs)
        embeddings = mean_pooling(outputs, inputs['attention_mask']).cpu().numpy()[0] # Удаление лишней размерности [0]
    return embeddings

In [78]:
embeddings = df['all_text'].apply(get_sentence_embedding).tolist()

In [79]:
with open("./files/embeddings", "wb") as fp:
    pickle.dump(embeddings, fp)

In [76]:
type(embeddings)

pandas.core.series.Series

In [82]:
def search_movie(query, top_k=5, year=None):
    query_embedding = get_sentence_embedding(query)
    cos_scores = torch.nn.functional.cosine_similarity(torch.tensor(query_embedding), torch.tensor(embeddings))
    df['similarity'] = cos_scores.tolist()
    res = df.sort_values(by='similarity', ascending=False)
    if year:
        res = res[res['year'] == year]
    return res.head(top_k)

In [80]:
import streamlit as st
import spacy
import pymorphy3


@st.cache_data
def load_models():
    # Создание объекта для морфологического анализа
    morph = pymorphy3.MorphAnalyzer()
    # Загрузка модели spaCy для русского языка
    nlp = spacy.load("ru_core_news_lg")
    return nlp, morph


def sort_by_entities(df: pd.DataFrame, text: str, morph: pymorphy3.analyzer.MorphAnalyzer, nlp):
    genres = {'аниме',
              'биография',
              'боевик',
              'вестерн',
              'военный',
              'детектив',
              'детский',
              'документальный',
              'драма',
              'исторический',
              'комедия',
              'короткометражный',
              'криминал',
              'мелодрама',
              'музыкальный',
              'мультфильмы',
              'мюзикл',
              'приключения',
              'семейный',
              'спорт',
              'триллер',
              'ужасы',
              'фантастика',
              'фэнтези',
              'эротика'}
    # Обработка текста
    doc = nlp(text)

    # Извлечение сущностей
    entities = [entity.text for entity in doc.ents if entity.label_ == "PER"]

    persons = []
    for entity in entities:
        persons.append(" ".join([morph.parse(person)[0].normal_form for person in entity.split()]))

    conditions = []
    for person in persons:
        for word in person.split(" "):
            if len(word) > 3:
                word = word[:-1]
            conditions.append(df["actors"].str.contains(word, na=False, case=False))
            conditions.append(df["director"].str.contains(word, na=False, case=False))

    combined_condition = pd.Series([False] * len(df), index=df.index)
    if len(conditions) > 1:
        combined_condition = conditions[0]
        for condition in conditions[1:]:
            combined_condition |= condition

    search_genre = []
    for genre in genres:
        if genre in text.lower():
            search_genre.append(df[genre] == 1)

    if len(search_genre) > 0:
        for condition in search_genre:
            combined_condition |= condition

    if len(search_genre) + len(persons) > 0:
        filtered = pd.concat([df[combined_condition], df[~combined_condition]])
    else:
        filtered = df
    return filtered


2025-02-28 16:33:26.797 No runtime found, using MemoryCacheStorageManager


In [84]:
query = 'фильм про любовь и приключения c Лайа Коста'
# query = 'комедия про детей'
result = search_movie(query)
nlp, morph = load_models()
result_new = sort_by_entities(result, query, morph, nlp)
display(result[['title', 'year', 'actors', 'description', 'similarity']])

,title,year,actors,description,similarity
26014,Просто любовь,2023,"Лайа Коста, Ховик Кеучкерян, Ингрид Гарсия Йон...",Тридцатилетняя Нэт сбегает от напряженной горо...,0.780175
40107,Драма/Мекс,2006,"Фернандо Бесерриль, Мириана Моро, Хуан Пабло К...","Жизненная мелодрама, в центре сюжета которой л...",0.757315
9000,Двое,1961,"Беба Лончар, Миха Балох, Нада Касапич, Борисла...","Молодой человек влюбляется в девушку, которую ...",0.744767
7779,Говори мне о любви,1986,"Фернанда Торрес, Талес Пан Шакон","Чувственная бразильская драма, герои которой —...",0.739419
38213,Противоположность любви,2011,"Адриана Угарте, Уго Сильва, Луис Кальехо, Март...",После нескольких неудач в любви Мерче и Рауль ...,0.731924


In [75]:
type(result)

pandas.core.frame.DataFrame

In [24]:
torch.cuda.empty_cache()